In [ ]:
!pip install numpy
!pip install pandas
!pip install geopandas

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import gdutils.extract as et
import gdutils.dataqa as dq

## Data Retrieval

In [4]:
mggg_file = '../tests/inputs/CT_precincts/CT_precincts.shp'
medsl_file = '../tests/inputs/precinct_2018/precinct_2018.csv'

In [5]:
mggg_gdf = et.read_file(mggg_file).extract()
mggg_gdf.head()

,STATEFP10,COUNTYFP10,NAME10,NAMELSAD10,PRECINCT,AG18D,AG18R,COMP18D,COMP18R,GOV18D,...,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,CD,SEND,HDIST,geometry
0,09,001,Greenwich 5,Greenwich 5 Voting District,Greenwich - DISTRICT 5-Riverside School,509,638,455,670,494,...,1,0,53,0,2,5,04,036,150,"POLYGON ((235144.885 174777.456, 235144.885 17..."
1,09,001,Bethel 1,Bethel 1 Voting District,Bethel - DISTRICT 1-Bethel Municipal Center 1,629,445,637,421,625,...,55,3,137,0,15,18,05,026,107,"POLYGON ((249146.480 214327.498, 249166.545 21..."
2,09,001,Bethel 4,Bethel 4 Voting District,Bethel - DISTRICT 4-Bethel Municipal Center 4,1083,931,1105,877,1065,...,85,3,199,0,44,47,05,026,002,"POLYGON ((247626.684 211866.650, 247598.944 21..."
3,09,001,Monroe 3,Monroe 3 Voting District,Monroe - DISTRICT 3-Monroe Elementary School,621,939,606,923,575,...,45,3,35,1,2,18,04,022,112,"POLYGON ((264253.271 205546.637, 264238.502 20..."
4,09,001,Monroe 1,Monroe 1 Voting District,Monroe - DISTRICT 1-Fawn Hollow School,1278,1656,1294,1584,1233,...,54,1,108,1,1,31,04,021,112,"POLYGON ((264466.508 210942.339, 264481.379 21..."


In [5]:
medsl_all_gdf = et.read_file(medsl_file).extract()
medsl_all_gdf.head()

/usr/local/lib/python3.8/site-packages/gdutils/extract.py:193: DtypeWarning: Columns (0,2,8,14) have mixed types.Specify dtype option on import or set low_memory=False.
  self.infile = infile


,precinct,office,party,mode,votes,jurisdiction,county,candidate,district,dataverse,year,stage,state,special,writein,state_po,state_fips,state_cen,state_ic,geometry
0,10 JONES COMMUNITY CTR,Straight Party,democratic,election day,98,Autauga,Autauga,Alabama Democratic Party,NaN,all,2018,gen,Alabama,False,False,AL,1,63,41,None
1,10 JONES COMMUNITY CTR,Straight Party,republican,election day,110,Autauga,Autauga,Alabama Republican Party,NaN,all,2018,gen,Alabama,False,False,AL,1,63,41,None
2,10 JONES COMMUNITY CTR,US House,democratic,election day,118,Autauga,Autauga,Tabitha Isner,2,house,2018,gen,Alabama,False,False,AL,1,63,41,None
3,10 JONES COMMUNITY CTR,US House,republican,election day,153,Autauga,Autauga,Martha Roby,2,house,2018,gen,Alabama,False,False,AL,1,63,41,None
4,10 JONES COMMUNITY CTR,US House,NaN,election day,0,Autauga,Autauga,NaN,2,house,2018,gen,Alabama,False,True,AL,1,63,41,None


In [6]:
medsl_ct_gdf = et.ExtractTable(medsl_all_gdf, column='state', value='Connecticut').extract()
medsl_ct_gdf.head()

,precinct,office,party,mode,votes,jurisdiction,county,candidate,district,dataverse,year,stage,special,writein,state_po,state_fips,state_cen,state_ic,geometry
state,,,,,,,,,,,,,,,,,,,
Connecticut,Andover Town Hall,Governor and Lieutenant Governor,democrat,total,618,Andover,Tolland,Ned Lamont and Susan Bysiewicz,statewide,state,2018,gen,False,False,CT,9,16,1,None
Connecticut,Andover Town Hall,Governor and Lieutenant Governor,republican,total,863,Andover,Tolland,Bob Stefanowski and Joe Markley,statewide,state,2018,gen,False,False,CT,9,16,1,None
Connecticut,Andover Town Hall,Governor and Lieutenant Governor,working families party,total,19,Andover,Tolland,Ned Lamont and Susan Bysiewicz,statewide,state,2018,gen,False,False,CT,9,16,1,None
Connecticut,Andover Town Hall,Governor and Lieutenant Governor,independent,total,29,Andover,Tolland,Bob Stefanowski and Joe Markley,statewide,state,2018,gen,False,False,CT,9,16,1,None
Connecticut,Andover Town Hall,Governor and Lieutenant Governor,libertarian,total,6,Andover,Tolland,Rodney Hanscomb and Jeffrey Thibeault,statewide,state,2018,gen,False,False,CT,9,16,1,None


### Data Wrangling MEDSL Data

In [8]:
medsl_pvt_df = medsl_ct_gdf.pivot_table(index='precinct', 
                                        columns=['office', 'party'], 
                                        values='votes')
medsl_pvt_df.columns = [' '.join(col).strip() for col in medsl_pvt_df.columns.values]
    # TODO: consider making a pivot-flatten function for extract module?

In [9]:
medsl_gdf = et.ExtractTable(medsl_pvt_df).extract()
medsl_gdf.head()

,Attorney General democrat,Attorney General green,Attorney General independent,Attorney General republican,Attorney General working families party,Comptroller democrat,Comptroller green,Comptroller independent,Comptroller libertarian,Comptroller republican,...,US House independent,US House libertarian,US House republican,US House working families party,US Senate democrat,US Senate green,US Senate libertarian,US Senate republican,US Senate working families party,geometry
precinct,,,,,,,,,,,,,,,,,,,,,
1 Town Hall,431.0,22.0,35.0,702.0,26.0,494.0,15.0,35.0,20.0,590.0,...,NaN,47.0,478.0,69.0,558.0,10.0,15.0,584.0,49.0,None
1a Town Hall,361.0,12.0,31.0,579.0,9.0,431.0,10.0,18.0,13.0,485.0,...,NaN,44.0,355.0,59.0,472.0,1.0,15.0,471.0,37.0,None
2 Central Village Fire Station,194.0,8.0,27.0,389.0,6.0,225.0,11.0,15.0,10.0,338.0,...,NaN,27.0,253.0,42.0,259.0,8.0,7.0,333.0,29.0,None
3 Moosup Fire Station,557.0,20.0,66.0,937.0,31.0,658.0,17.0,42.0,37.0,776.0,...,NaN,75.0,609.0,93.0,733.0,13.0,28.0,785.0,59.0,None
330 Windsor Avenue,1708.0,24.0,23.0,243.0,36.0,1697.0,18.0,15.0,15.0,212.0,...,NaN,NaN,200.0,55.0,1779.0,11.0,9.0,216.0,51.0,None


## Aggregation Comparisons